# **空調制御データ分析 / AC Control Data Analysis**

**目的 (Objective):**  
顧客の空調制御および電力消費データを分析・可視化し、HVAC制御ロジックの計画や評価を支援する。  
この作業はJupyter Notebook内で再現可能かつ体系的に行い、英語と日本語の両方で記録する。  

**Objective:**  
Analyze and visualize customer AC control and power consumption data  
to support planning and evaluation of HVAC control logic.  
The work should be reproducible, structured in a Jupyter Notebook,  
and documented in both English and Japanese.

1. セットアップとインポート / Setup and Imports  
2. データ読み込みと確認 / Load and Inspect  
3. サニティチェックとクリーニング / Sanity Checks and Cleaning  
4. 周波数の正規化 / Regularize Frequency  
5. 特徴量エンジニアリング / Feature Engineering  
6. KPIと可視化 / KPIs and Visualization  
7. 結果のまとめ / Summary of Findings


## エグゼクティブサマリー / Executive Summary


## セットアップとインポート / Setup and Imports


In [ ]:
# セットアップとインポート
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from pylab import rcParams
import os, sys, pathlib, warnings, logging
from typing import Dict, Tuple
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.tseries.frequencies import to_offset
import time
from xgboost import XGBRegressor, plot_importance
from sklearn.model_selection import train_test_split

# ロギング設定
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
warnings.filterwarnings("ignore")

print("📦 ライブラリ読み込み完了")
print("=" * 60)

In [ ]:
# ストア名とデータパスの設定
STORE_NAME = "Clea"  # 例: "IsetanMitsukoshi"
DATA_ROOT = pathlib.Path(r"C:\Users\maxtr\OneDrive\Arbeitarbeit\Input") 
MASTER_DATA_PATH = pathlib.Path(r"C:\Users\maxtr\OneDrive\Arbeitarbeit\Master")
PROCESSED_DATA_PATH = pathlib.Path(r"C:\Users\maxtr\OneDrive\Arbeitarbeit\Processed") 

print(f"🏪 分析対象ストア: {STORE_NAME}")
print(f"📁 データルート: {DATA_ROOT}")
print(f"📋 マスタデータ: {MASTER_DATA_PATH}")
print(f"🔧 前処理済みデータ: {PROCESSED_DATA_PATH}")

# パスの存在確認
print(f"\n📊 パス存在確認:")
print(f"   データルート: {DATA_ROOT.exists()}")
print(f"   マスタデータ: {MASTER_DATA_PATH.exists()}")
print(f"   前処理済みデータ: {PROCESSED_DATA_PATH.exists()}")

In [ ]:
# CSVファイルを結合して読み込む関数
def read_all_csv(folder: pathlib.Path, pattern="*.csv") -> pd.DataFrame:
    """フォルダ内のCSVを結合"""
    dfs = []
    for fp in folder.rglob(pattern):
        try:
            df = pd.read_csv(fp)
            df["__source_file"] = fp.name
            dfs.append(df)
        except Exception as e:
            logging.warning(f"読み込み失敗: {fp}: {e}")
    
    if not dfs:
        logging.warning(f"{folder} にCSVファイルが見つかりません。")
        return pd.DataFrame()
    
    result = pd.concat(dfs, ignore_index=True)
    logging.info(f"{len(dfs)}ファイル、合計{len(result):,}行を読み込み完了")
    return result

print("📋 データ読み込み関数定義完了")

## Defining Color Template

In [ ]:
# Seaborn設定関数
def init_setting():
    """グラフ表示設定"""
    sns.set_style("whitegrid", {"grid.linestyle": "--"})
    sns.set_context("paper", 1.5, {"lines.linewidth": 1.5})
    sns.set_palette("Set2")
    current_palette = list(
        sns.color_palette("Set2", n_colors=8) + sns.color_palette("Set3_r", n_colors=8)
    )
    sns.set_palette(current_palette)
    sns.palplot(current_palette)
    plt.show()

    # 日本語フォント設定
    rcParams["font.family"] = "sans-serif"
    rcParams["font.sans-serif"] = [
        "Yu Gothic", "Meirio", "Noto Sans CJK JP", "IPAexGothic"
    ]

init_setting()

In [ ]:
# Plotlyのカラーテンプレート設定
def set_plotly_palette():
    """Plotly用カラーテンプレート設定"""
    current_palette = list(
        sns.color_palette("Set2", n_colors=8) + sns.color_palette("Set3_r", n_colors=8)
    )
    hex_colors = [mcolors.to_hex(c) for c in current_palette]
    
    custom_template = dict(
        layout=go.Layout(
            colorway=hex_colors,
            title_font=dict(size=16, family="Yu Gothic, Noto Sans CJK JP, sans-serif"),
            font=dict(size=12, family="Yu Gothic, Noto Sans CJK JP, sans-serif"),
        )
    )
    pio.templates["custom"] = custom_template
    pio.templates.default = "custom"
    print("✅ Plotlyカラーテンプレート設定完了")

set_plotly_palette()

## データ読み込みと確認 / Load and Inspect


In [ ]:
# データ読み込み
print(f"📁 {STORE_NAME}のデータ読み込み開始")
print("=" * 60)
df_control = read_all_csv(DATA_ROOT / "ac-control")
df_power   = read_all_csv(DATA_ROOT / "ac-power-meter")

print(f"\n📊 読み込み結果:")
print(f"   AC制御データ: {df_control.shape}")
print(f"   電力メーターデータ: {df_power.shape}")

In [ ]:
# デバッグ: df_controlのカラム名を確認
print("🔍 AC制御データのカラム名確認:")
print("=" * 60)
print("df_controlのカラム名:")
print(df_control.columns.tolist())
print(f"\nカラム数: {len(df_control.columns)}")
print(f"データ形状: {df_control.shape}")

In [ ]:
# A/C Fan Speed列のNaNでない値を確認
if 'A/C Fan Speed' in df_control.columns:
    print("A/C Fan Speed not NaN count:", df_control["A/C Fan Speed"].notna().sum())
    print("A/C Fan Speed unique values:", df_control["A/C Fan Speed"].dropna().unique())
else:
    print("❌ 'A/C Fan Speed'カラムが見つかりません")
    print("利用可能なカラム:", df_control.columns.tolist())

The NaN values exist because only a few rows include a description of the fan speed in the raw dataset.


In [ ]:
# データの基本情報を表示
print("📊 データの基本情報:")
print("=" * 60)

# for name, df in [("AC制御", df_control), ("電力メーター", df_power), ("アクティブレベル", df_active), ("グループトリガー", df_trigger), ("テーブル", df_table)]:
for name, df in [("AC制御", df_control), ("電力メーター", df_power)]:
    if not df.empty:
        print(f"\n{name}データ:")
        print(f"  形状: {df.shape}")
        print(f"  カラム: {list(df.columns)}")
        print(f"  メモリ使用量: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        
        # 日時列の確認
        datetime_cols = [col for col in df.columns if 'Datetime' in col.lower() or '日時' in col]
        if datetime_cols:
            print(f"  日時列: {datetime_cols}")
            print(f"  期間: {df[datetime_cols[0]].min()} ～ {df[datetime_cols[0]].max()}")
    else:
        print(f"\n{name}データ: 空のデータフレーム")


In [ ]:
DEBUG = False

# Limit data to the first 1000 rows, not random sample
if DEBUG:
    df_control = df_control.head(100000)
    df_power   = df_power.head(100000)


## サニティチェックとクリーニング / Sanity Checks and Cleaning


In [ ]:
# データクリーニング関数の定義
def clean_ac_control(df):
    """AC制御データのクリーニング"""
    if df.empty:
        return df
    
    df_clean = df.copy()
    
    # 日時列の統一
    datetime_cols = [col for col in df_clean.columns if 'datetime' in col.lower() or '日時' in col]
    if datetime_cols:
        datetime_col = datetime_cols[0]
        df_clean[datetime_col] = pd.to_datetime(df_clean[datetime_col], utc=True)
        df_clean[datetime_col] = df_clean[datetime_col].dt.tz_localize(None)
        df_clean[datetime_col] = df_clean[datetime_col].dt.floor("T")
    
    # 外れ値除去（3σルール）
    numeric_cols = ["A/C Set Temperature", "Indoor Temp.", "Outdoor Temp."]
    for col in numeric_cols:
        if col in df_clean.columns:
            before_count = len(df_clean)
            mean_val = df_clean[col].mean()
            std_val = df_clean[col].std()
            threshold = 3.0 * std_val
            df_clean = df_clean[abs(df_clean[col] - mean_val) <= threshold]
            after_count = len(df_clean)
            print(f"   🔄 {col}外れ値除去: {before_count:,} → {after_count:,}件")
    
    # 欠損値処理（線形補間、湿度は除外）
    for col in numeric_cols:
        if col in df_clean.columns:
            before_nan = df_clean[col].isna().sum()
            df_clean[col] = df_clean[col].interpolate(method='linear')
            after_nan = df_clean[col].isna().sum()
            print(f"   🔄 {col}欠損値補完: {before_nan:,} → {after_nan:,}件")
    
    # 日付列の追加
    if datetime_cols:
        df_clean["date"] = df_clean[datetime_cols[0]].dt.date
        df_clean["datetime"] = df_clean[datetime_cols[0]]
    
    print(f"   ✅ AC制御データクリーニング完了: {len(df_clean):,}件")
    return df_clean

def clean_ac_power(df):
    """電力メーターデータのクリーニング"""
    if df.empty:
        return df
    
    df_clean = df.copy()
    
    # 日時列の統一
    datetime_cols = [col for col in df_clean.columns if 'datetime' in col.lower() or '日時' in col]
    if datetime_cols:
        datetime_col = datetime_cols[0]
        df_clean[datetime_col] = pd.to_datetime(df_clean[datetime_col], utc=True)
        df_clean[datetime_col] = df_clean[datetime_col].dt.tz_localize(None)
        df_clean[datetime_col] = df_clean[datetime_col].dt.floor("T")
    
    # 外れ値除去
    if "Phase A" in df_clean.columns:
        before_count = len(df_clean)
        mean_val = df_clean["Phase A"].mean()
        std_val = df_clean["Phase A"].std()
        threshold = 3.0 * std_val
        df_clean = df_clean[abs(df_clean["Phase A"] - mean_val) <= threshold]
        after_count = len(df_clean)
        print(f"   🔄 Phase A外れ値除去: {before_count:,} → {after_count:,}件")
    
    
    # 日付列の追加
    if datetime_cols:
        df_clean["date"] = df_clean[datetime_cols[0]].dt.date
        df_clean["datetime"] = df_clean[datetime_cols[0]]
    
    print(f"   ✅ 電力メーターデータクリーニング完了: {len(df_clean):,}件")
    return df_clean


print("🔧 データクリーニング関数定義完了")

In [ ]:
# データクリーニングの実行
print("🔧 データクリーニング実行中...")
print("=" * 60)

control_clean = clean_ac_control(df_control)
power_clean   = clean_ac_power(df_power)

print(f"\n📊 クリーニング後のデータ形状:")
print(f"   AC制御データ: {control_clean.shape}")
print(f"   電力メーターデータ: {power_clean.shape}")

In [ ]:
# クリーニング済みデータの表示
print("📋 クリーニング済みデータのサンプル:")
print("=" * 60)

display(control_clean.head())
display(power_clean.head())

## 周波数の正規化 / Regularize Frequency


In [ ]:
# 時別データの正規化と特徴量作成
print("🕐 時別データの正規化と特徴量作成中...")
print("=" * 60)

# 時間特徴量の追加
def add_time_features(df, datetime_col='datetime'):
    """時間特徴量を追加"""
    if df.empty or datetime_col not in df.columns:
        return df
    
    df_time = df.copy()
    df_time['hour'] = df_time[datetime_col].dt.hour
    df_time['day_of_week'] = df_time[datetime_col].dt.weekday
    df_time['day_name'] = df_time[datetime_col].dt.day_name()
    df_time['is_weekend'] = df_time['day_of_week'] >= 5
    df_time['is_business_hours'] = (df_time['hour'] >= 9) & (df_time['hour'] <= 18)
    df_time['month'] = df_time[datetime_col].dt.month
    df_time['day'] = df_time[datetime_col].dt.day
    
    return df_time

# AC制御データに時間特徴量を追加
if not control_clean.empty:
    control_clean = add_time_features(control_clean)
    print(f"✅ AC制御データに時間特徴量を追加: {control_clean.shape}")

# 電力データに時間特徴量を追加
if not power_clean.empty:
    power_clean = add_time_features(power_clean)
    print(f"✅ 電力データに時間特徴量を追加: {power_clean.shape}")

print("🕐 時間特徴量追加完了")

## 特徴量エンジニアリング / Feature Engineering


In [ ]:
# 時別集計による特徴量作成
print("🔧 時別集計による特徴量作成中...")
print("=" * 60)

if not control_clean.empty:
    # AC制御データの時別集計
    print("📊 AC制御データの時別集計中...")
    ac_hourly = control_clean.groupby(['datetime', 'A/C Name']).agg({
    'A/C Set Temperature': ['mean', 'std', 'min', 'max'],
    'Indoor Temp.': ['mean', 'std', 'min', 'max'],
    'Outdoor Temp.': ['mean', 'std', 'min', 'max'],
    'A/C ON/OFF': ['count', 'nunique'],   # oder später Mapping ON=1/OFF=0
    'A/C Mode': ['first', pd.Series.mode],
    'A/C Fan Speed': ['first', pd.Series.mode]
    }).reset_index()

    
    # カラム名を平坦化
    ac_hourly.columns = ['datetime', 'A/C Name'] + [f"{col[0]}_{col[1]}" for col in ac_hourly.columns[2:]]
    
    # 時間特徴量を追加
    ac_hourly = add_time_features(ac_hourly)
    
    # ラグ特徴量の追加
    ac_hourly = ac_hourly.sort_values(['A/C Name', 'datetime'])
    lag_cols = ['A/C Set Temperature_mean', 'Indoor Temp._mean', 'Outdoor Temp._mean']
    for col in lag_cols:
        if col in ac_hourly.columns:
            ac_hourly[f"{col}_lag1"] = ac_hourly.groupby('A/C Name')[col].shift(1)
    
    print(f"✅ AC制御時別集計完了: {len(ac_hourly):,}件")
    print(f"📋 特徴量カラム: {list(ac_hourly.columns)}")

if not power_clean.empty:
    # 電力データの時別集計
    print("📊 電力データの時別集計中...")
    power_hourly = power_clean.groupby('datetime').agg({
        'Phase A': ['mean', 'std', 'min', 'max', 'sum']
    }).reset_index()
    
    # カラム名を平坦化
    power_hourly.columns = ['datetime'] + [f"power_{col[1]}" for col in power_hourly.columns[1:]]
    
    # 時間特徴量を追加
    power_hourly = add_time_features(power_hourly)
    
    print(f"✅ 電力時別集計完了: {len(power_hourly):,}件")
    print(f"📋 特徴量カラム: {list(power_hourly.columns)}")

print("🔧 特徴量作成完了")

## KPIと可視化 / KPIs and Visualization


In [ ]:
print("📋 分析結果サマリー（簡略版）")
print("=" * 60)
print(f"🏪 ストア: {STORE_NAME}")
print(f"📅 期間: {control_clean['datetime'].min()} ～ {control_clean['datetime'].max()}")
print(f"📊 データ件数: {len(control_clean):,}")

if 'A/C Set Temperature' in control_clean.columns:
    print(f"平均設定温度: {control_clean['A/C Set Temperature'].mean():.1f}°C")
if 'Indoor Temp.' in control_clean.columns:
    print(f"平均室内温度: {control_clean['Indoor Temp.'].mean():.1f}°C")
if 'Outdoor Temp.' in control_clean.columns:
    print(f"平均外気温度: {control_clean['Outdoor Temp.'].mean():.1f}°C")
if 'Phase A' in power_clean.columns:
    print(f"平均電力消費: {power_clean['Phase A'].mean():.2f}kW")
    print(f"最大電力消費: {power_clean['Phase A'].max():.2f}kW")

In [ ]:
print("📈 高速版：時間別温度推移グラフ（曜日表示付き）")
print("=" * 60)

if not control_clean.empty and 'datetime' in control_clean.columns:

    # データを軽量化 
    df_plot = control_clean.copy()
    df_plot = df_plot.set_index('datetime').resample('1H').mean(numeric_only=True).reset_index()
    print(f"📉 プロット用データ: {len(df_plot):,}件 (1時間平均)")

    # グラフ作成 
    fig = go.Figure()

    # 設定温度
    if 'A/C Set Temperature' in df_plot.columns:
        fig.add_trace(go.Scatter(
            x=df_plot['datetime'],
            y=df_plot['A/C Set Temperature'],
            mode='lines',
            name='設定温度',
            line=dict(color='red', width=1.8)
        ))

    # 室内温度
    if 'Indoor Temp.' in df_plot.columns:
        fig.add_trace(go.Scatter(
            x=df_plot['datetime'],
            y=df_plot['Indoor Temp.'],
            mode='lines',
            name='室内温度',
            line=dict(color='blue', width=1.8)
        ))

    # 外気温度
    if 'Outdoor Temp.' in df_plot.columns:
        fig.add_trace(go.Scatter(
            x=df_plot['datetime'],
            y=df_plot['Outdoor Temp.'],
            mode='lines',
            name='外気温度',
            line=dict(color='green', width=1.5)
        ))

    fig.update_layout(
        title=f'{STORE_NAME} - 温度推移（曜日表示付き・高速版）',
        xaxis_title='日時',
        yaxis_title='温度 (°C)',
        hovermode='x unified',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        height=500,
        template='plotly_white'
    )

    weekend_mask = df_plot['datetime'].dt.weekday >= 5
    if weekend_mask.any():
        weekend_x = df_plot.loc[weekend_mask, 'datetime']
        if not weekend_x.empty:
            x0, x1 = weekend_x.min(), weekend_x.max()
            fig.add_vrect(
                x0=x0, x1=x1,
                fillcolor="rgba(255,0,0,0.05)",
                line_width=0,
                layer="below",
                annotation_text="週末",
                annotation_position="top left"
            )

    fig.show()

In [ ]:
# 運転モードと風量の分析
print("⚙️ 運転モードと風量の分析")
print("=" * 60)

if not control_clean.empty:
    # 運転モードの分析
    print("📊 運転モードの分析:")
    if 'A/C Mode' in control_clean.columns:
        mode_counts = control_clean['A/C Mode'].value_counts()
        print(mode_counts)
        
        # 運転モードの分布
        fig = px.pie(
            values=mode_counts.values,
            names=mode_counts.index,
            title=f'{STORE_NAME} - 運転モードの分布'
        )
        fig.show()
    
    # 風量の分析
    print("\n📊 風量の分析:")
    if 'A/C Fan Speed' in control_clean.columns:
        fan_speed_counts = control_clean['A/C Fan Speed'].value_counts()
        print(fan_speed_counts)
        
        # 風量の分布
        fig = px.bar(
            x=fan_speed_counts.index,
            y=fan_speed_counts.values,
            title=f'{STORE_NAME} - 風量の分布',
            labels={'x': '風量', 'y': '件数'}
        )
        fig.show()
    
    # 運転状態の分析
    print("\n📊 運転状態の分析:")
    if 'A/C ON/OFF' in control_clean.columns:
        # Strings in 0/1 umwandeln
        control_clean['ON_OFF_Binary'] = control_clean['A/C ON/OFF'].map(
        {'OFF': 0, 'ON': 1}
        )

        on_off_counts = control_clean['ON_OFF_Binary'].value_counts()
        print(on_off_counts)

        fig = px.pie(
            values=on_off_counts.values,
            names=['OFF' if x == 0 else 'ON' for x in on_off_counts.index],
            title=f'{STORE_NAME} - 運転状態の分布'
        )
        fig.show()

In [ ]:
# 時間別・曜日別の運転パターン分析
print("🕐 時間別・曜日別の運転パターン分析")
print("=" * 60)

if not control_clean.empty and 'hour' in control_clean.columns and 'day_name' in control_clean.columns:
    
    # ON/OFF を数値化 (ON=1, OFF=0)
    if 'A/C ON/OFF' in control_clean.columns:
        control_clean['A/C ON/OFF_num'] = control_clean['A/C ON/OFF'].map({'ON': 1, 'OFF': 0})
    else:
        print("❌ 'A/C ON/OFF' 列が存在しません")
    
    # 時間別運転率
    if 'A/C ON/OFF_num' in control_clean.columns:
        hourly_operation = control_clean.groupby('hour')['A/C ON/OFF_num'].mean()
        
        if not hourly_operation.empty:
            fig = px.bar(
                x=hourly_operation.index,
                y=hourly_operation.values,
                title=f'{STORE_NAME} - 時間別運転率',
                labels={'x': '時間', 'y': '運転率'}
            )
            fig.update_layout(height=400)
            fig.show()
    
    # 曜日別運転率
    if 'A/C ON/OFF_num' in control_clean.columns:
        daily_operation = control_clean.groupby('day_name')['A/C ON/OFF_num'].mean()
        
        if not daily_operation.empty:
            # 曜日の順序を設定
            day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
            daily_operation = daily_operation.reindex(day_order)
            
            fig = px.bar(
                x=daily_operation.index,
                y=daily_operation.values,
                title=f'{STORE_NAME} - 曜日別運転率',
                labels={'x': '曜日', 'y': '運転率'}
            )
            fig.update_layout(height=400)
            fig.show()
    
    # 時間別・曜日別のヒートマップ
    if 'A/C ON/OFF_num' in control_clean.columns:
        pivot_data = control_clean.pivot_table(
            values='A/C ON/OFF_num', 
            index='day_name', 
            columns='hour', 
            aggfunc='mean'
        )
        
        # 曜日の順序を設定
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        pivot_data = pivot_data.reindex(day_order)
        
        fig = px.imshow(
            pivot_data.values,
            x=pivot_data.columns,
            y=pivot_data.index,
            color_continuous_scale='Blues',
            title=f'{STORE_NAME} - 時間別・曜日別運転率ヒートマップ',
            labels={'x': '時間', 'y': '曜日', 'color': '運転率'}
        )
        fig.update_layout(height=500)
        fig.show()

In [ ]:
# 電力消費分析
print("⚡ 電力消費分析")
print("=" * 60)

if not power_clean.empty:
    # 電力消費の基本統計
    print("📊 電力消費の基本統計:")
    if 'Phase A' in power_clean.columns:
        power_stats = power_clean['Phase A'].describe()
        print(power_stats)
        
        # 電力消費の分布
        fig = px.histogram(
            power_clean, 
            x='Phase A', 
            nbins=50,
            title=f'{STORE_NAME} - 電力消費の分布',
            labels={'Phase A': '電力消費 (kW)', 'count': '件数'}
        )
        fig.show()
    
    # 時間別電力消費
    if 'hour' in power_clean.columns and 'Phase A' in power_clean.columns:
        hourly_power = power_clean.groupby('hour')['Phase A'].mean()
        
        fig = px.bar(
            x=hourly_power.index,
            y=hourly_power.values,
            title=f'{STORE_NAME} - 時間別平均電力消費',
            labels={'x': '時間', 'y': '平均電力消費 (kW)'}
        )
        fig.update_layout(height=400)
        fig.show()
    
    # 曜日別電力消費
    if 'day_name' in power_clean.columns and 'Phase A' in power_clean.columns:
        daily_power = power_clean.groupby('day_name')['Phase A'].mean()
        
        # 曜日の順序を設定
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        daily_power = daily_power.reindex(day_order)
        
        fig = px.bar(
            x=daily_power.index,
            y=daily_power.values,
            title=f'{STORE_NAME} - 曜日別平均電力消費',
            labels={'x': '曜日', 'y': '平均電力消費 (kW)'}
        )
        fig.update_layout(height=400)
        fig.show()

## 結果のまとめ / Summary of Findings


In [ ]:
# 分析結果のサマリー
print("📋 分析結果サマリー")
print("=" * 60)

print(f"🏪 分析対象ストア: {STORE_NAME}")
print(f"📅 分析期間: {control_clean['datetime'].min()} ～ {control_clean['datetime'].max()}")
print(f"📊 総データ件数: {len(control_clean):,}件")

# 主要KPI
if not control_clean.empty:
    print(f"\n🌡️ 温度関連KPI:")
    if 'A/C Set Temperature' in control_clean.columns:
        print(f"   平均設定温度: {control_clean['A/C Set Temperature'].mean():.1f}°C")
        print(f"   設定温度範囲: {control_clean['A/C Set Temperature'].min():.1f}°C ～ {control_clean['A/C Set Temperature'].max():.1f}°C")
    
    if 'Indoor Temp.' in control_clean.columns:
        print(f"   平均室内温度: {control_clean['Indoor Temp.'].mean():.1f}°C")
        print(f"   室内温度範囲: {control_clean['Indoor Temp.'].min():.1f}°C ～ {control_clean['Indoor Temp.'].max():.1f}°C")
    
    if 'Outdoor Temp.' in control_clean.columns:
        print(f"   平均外気温度: {control_clean['Outdoor Temp.'].mean():.1f}°C")
        print(f"   外気温度範囲: {control_clean['Outdoor Temp.'].min():.1f}°C ～ {control_clean['Outdoor Temp.'].max():.1f}°C")
    
    print(f"\n⚙️ 運転関連KPI:")
    if 'A/C ON/OFF' in control_clean.columns:
        # ON/OFFを数値化して運転率を計算 (ON=1, OFF=0)
        if 'A/C ON/OFF_num' not in control_clean.columns:
            control_clean['A/C ON/OFF_num'] = control_clean['A/C ON/OFF'].map({'ON': 1, 'OFF': 0})
        operation_rate = control_clean['A/C ON/OFF_num'].mean()
        print(f"   運転率: {operation_rate:.1%}")
    
    if 'A/C Mode' in control_clean.columns:
        most_common_mode = control_clean['A/C Mode'].mode().iloc[0] if not control_clean['A/C Mode'].mode().empty else "N/A"
        print(f"   最頻出運転モード: {most_common_mode}")
    
    if 'A/C Fan Speed' in control_clean.columns:
        most_common_fan = control_clean['A/C Fan Speed'].mode().iloc[0] if not control_clean['A/C Fan Speed'].mode().empty else "N/A"
        print(f"   最頻出風量: {most_common_fan}")

if not power_clean.empty and 'Phase A' in power_clean.columns:
    print(f"\n⚡ 電力関連KPI:")
    print(f"   平均電力消費: {power_clean['Phase A'].mean():.2f}kW")
    print(f"   最大電力消費: {power_clean['Phase A'].max():.2f}kW")
    print(f"   電力消費範囲: {power_clean['Phase A'].min():.2f}kW ～ {power_clean['Phase A'].max():.2f}kW")

print(f"\n✅ 分析完了: {STORE_NAME}の空調制御データ分析が完了しました")

# Feature Importance

## Creating new Dataframe

In [ ]:
# ON/OFFを数値に変換
control_clean['A/C ON/OFF_num'] = control_clean['A/C ON/OFF'].map({'ON': 1, 'OFF': 0})

# 運転モードと風量をカテゴリコードに変換（数値化）
for col in ['A/C Mode', 'A/C Fan Speed']:
    if col in control_clean.columns:
        control_clean[col + '_enc'] = control_clean[col].astype('category').cat.codes

# 必要な列のみを選択
control_features = control_clean[['Datetime', 'A/C ON/OFF_num', 'A/C Mode_enc',
                                  'A/C Fan Speed_enc', 'A/C Set Temperature',
                                  'Indoor Temp.', 'Outdoor Temp.']]

# 電力データと結合（datetimeで結合）
df_model = pd.merge(control_features, power_clean[['Datetime', 'Phase A']],
                    on='Datetime', how='inner')

# 列名を統一
df_model.rename(columns={'Phase A': 'Power'}, inplace=True)

print("✅ 学習用データセットを作成しました。")
display(df_model.head())

In [ ]:
print("⚙️ モデル学習準備中...")

# 特徴量と目的変数を定義（カラム名の確認）
cols_expected = ['Datetime', 'A/C ON/OFF_num', 'A/C Mode_enc', 
                 'A/C Fan Speed_enc', 'A/C Set Temperature', 
                 'Indoor Temp.', 'Outdoor Temp.', 'Power']

missing_cols = [c for c in cols_expected if c not in df_model.columns]
if missing_cols:
    print(f"⚠️ 欠落カラム: {missing_cols}")
else:
    print("✅ すべてのカラムが存在します。")

# メモリ削減のための型変換
dtype_map = {
    'A/C ON/OFF_num': 'int8',
    'A/C Mode_enc': 'int8',
    'A/C Fan Speed_enc': 'int8',
    'A/C Set Temperature': 'float32',
    'Indoor Temp.': 'float32',
    'Outdoor Temp.': 'float32',
    'Power': 'float32'
}

df_model = df_model.astype(dtype_map)

# データをサンプリング（10万行まで）
df_model_sample = df_model.sample(n=min(100000, len(df_model)), random_state=42)

print(f"📊 サンプルデータ件数: {len(df_model_sample):,}")

# 特徴量と目的変数を定義
X = df_model_sample.drop(columns=['Datetime', 'Power'])
y = df_model_sample['Power']

# 学習用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# モデルの定義と学習
model = XGBRegressor(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1  # 全CPUを使用
)

print("🚀 モデル学習を開始します...")
model.fit(X_train, y_train)
print("✅ モデル学習完了！")

# 特徴重要度の可視化
plt.figure(figsize=(8, 5))
plot_importance(model, importance_type='gain')
plt.title("特徴重要度（電力予測）")
plt.xlabel("重要度（gain）")
plt.ylabel("特徴量")
plt.show()

print(f"📈 学習データ件数: {len(df_model_sample):,}")

In [ ]:
print("可視化を開始します...")

# メモリ負荷を軽減するためにサンプリング
df_viz = df_model.sample(n=min(100000, len(df_model)), random_state=42)

# 室温と電力の関係（運転状態別）
fig1 = px.scatter(
    df_viz,
    x='Indoor Temp.',
    y='Power',
    color=df_viz['A/C ON/OFF_num'].map({1: '運転中', 0: '停止中'}),
    title="室温と電力の関係（運転状態別）",
    labels={
        'Indoor Temp.': '室内温度 (°C)',
        'Power': '電力消費 (kW)',
        'color': '運転状態'
    }
)
fig1.update_traces(marker=dict(size=5, opacity=0.6))
fig1.update_layout(height=500)
fig1.show()


# 運転モード別の電力消費
fig2 = px.box(
    df_viz,
    x='A/C Mode_enc',
    y='Power',
    title="運転モード別の電力消費",
    labels={
        'A/C Mode_enc': '運転モード（数値化）',
        'Power': '電力消費 (kW)'
    }
)
fig2.update_layout(height=500)
fig2.show()


# 設定温度と電力の関係
fig3 = px.scatter(
    df_viz,
    x='A/C Set Temperature',
    y='Power',
    color='A/C Mode_enc',
    title="設定温度と電力の関係（モード別）",
    labels={
        'A/C Set Temperature': '設定温度 (°C)',
        'Power': '電力消費 (kW)',
        'color': '運転モード（数値化）'
    }
)
fig3.update_traces(marker=dict(size=5, opacity=0.6))
fig3.update_layout(height=500)
fig3.show()


# 風量設定別の電力消費
fig4 = px.box(
    df_viz,
    x='A/C Fan Speed_enc',
    y='Power',
    title="風量設定別の電力消費",
    labels={
        'A/C Fan Speed_enc': '風量設定（数値化）',
        'Power': '電力消費 (kW)'
    }
)
fig4.update_layout(height=500)
fig4.show()


# 設定温度と室温の関係
fig5 = px.scatter(
    df_viz,
    x='A/C Set Temperature',
    y='Indoor Temp.',
    color='A/C Mode_enc',
    title="設定温度と室温の関係（モード別）",
    labels={
        'A/C Set Temperature': '設定温度 (°C)',
        'Indoor Temp.': '室内温度 (°C)',
        'color': '運転モード（数値化）'
    }
)
fig5.update_traces(marker=dict(size=5, opacity=0.6))
fig5.update_layout(height=500)
fig5.show()